# **CHETNIPHAT PONGSRITHONE 6214022043**

# **1. Load libraries and data**

In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)


In [2]:
import pythainlp
import gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_least_angle.py:169: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not 

# **2. Download file**

In [3]:
import pandas as pd
df = pd.read_csv('Wongnai Reviews - Small.csv')
df

,Review ID,Review
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...
3,4,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...
...,...,...
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...


# **3. Tokenize Words**

In [4]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')' , '           ','-']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [5]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [6]:
df

,Review ID,Review,Review_tokenized
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...,"คน,ชอบ,ทาน,Macchiato,เป็นประจำ,นึง,เดิน,เข้าไป..."
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...,"Art,of,Coffee,Kasetsart,ร้านกาแฟ,รสชาติ,เยี่ยม..."
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...,"กวง,ทะเล,เผา,อาหารทะเล,เค้า,สด,เนื้อ,ปู,หวาน,ค..."
3,4,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...,"มีโอกาส,ตื่น,เช้า,โอกาส,มาหา,อร่อย,ทาน,ตอนเช้า..."
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...,"ชอบ,ทาน,กิน,อาหาร,เวียดนาม,บ้าน,ซอย,ราชครู,เข้..."
...,...,...,...
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."


# **4. Create Dictionary**

In [7]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [8]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', 'พลาด'

In [9]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

# **5. Topic Modeling**

In [23]:
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 7.07 s, sys: 95.3 ms, total: 7.17 s
Wall time: 7.12 s


In [24]:
pyLDAvis.gensim_models.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.019180 -0.010209       1        1  36.251698
1     -0.019095 -0.010346       2        1  35.509089
0     -0.000085  0.020555       3        1  28.239213, topic_info=            Term        Freq       Total Category  logprob  loglift
67            กก   77.000000   77.000000  Default  30.0000  30.0000
317            "   80.000000   80.000000  Default  29.0000  29.0000
462           ผม  185.000000  185.000000  Default  28.0000  28.0000
2           กาแฟ  306.000000  306.000000  Default  27.0000  27.0000
662            :  118.000000  118.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
825  เครื่องดื่ม   34.086320  119.209380   Topic3  -5.6679   0.0125
146         หวาน   44.192330  200.420729   Topic3  -5.4082  -0.2474
35            ชา   49.235760  248.265170   Topic3  -5.3002  -0.3534
731         นะคะ   36.246299  150.521504   Topic3  -5.6064  -0.1593
170         เค้า   32.701765  131.821005   Topic3  -5.7094  -0.1296

[251 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3690      1  0.245578         
3690      2  0.736733         
317       1  0.259774        "
317       2  0.309255        "
317       3  0.420586        "
...     ...       ...      ...
915       2  0.231523  ไต้หวัน
915       3  0.115762  ไต้หวัน
1532      1  0.117877        “
1532      2  0.707264        “
1532      3  0.235755        “

[424 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [25]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [26]:
df

,Review ID,Review,Review_tokenized,topics,score
0,1,เป็นคนที่ชอบทาน Macchiato เป็นประจำ มีวันนึงเด...,"คน,ชอบ,ทาน,Macchiato,เป็นประจำ,นึง,เดิน,เข้าไป...",2,0.998018
1,2,Art of Coffee Kasetsart เป็นร้านกาแฟรสชาติเยี่...,"Art,of,Coffee,Kasetsart,ร้านกาแฟ,รสชาติ,เยี่ยม...",2,0.998463
2,3,กวงทะเลเผา อาหารทะเลเค้าสดจริงๆเนื้อปูหวานไม่ค...,"กวง,ทะเล,เผา,อาหารทะเล,เค้า,สด,เนื้อ,ปู,หวาน,ค...",2,0.999749
3,4,วันนี้มีโอกาสตื่นเช้าครับเลยถึงโอกาสออกมาหาอะไ...,"มีโอกาส,ตื่น,เช้า,โอกาส,มาหา,อร่อย,ทาน,ตอนเช้า...",1,0.999085
4,5,ชอบมาทานร้านนี้ถ้าอยากกินอาหารเวียดนามใกล้บ้าน...,"ชอบ,ทาน,กิน,อาหาร,เวียดนาม,บ้าน,ซอย,ราชครู,เข้...",0,0.998906
...,...,...,...,...,...
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...",2,0.999671
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...",2,0.999133
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...",2,0.998839
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...",0,0.999546


In [38]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [39]:
get_lda_topics(model, num_topics)

,Topic # 01,Topic # 02,Topic # 03
0,อร่อย,อร่อย,อร่อย
1,ทาน,ทาน,รสชาติ
2,กาแฟ,รสชาติ,ดี
3,รสชาติ,ดี,ทาน
4,ดี,กิน,กิน
5,กิน,กาแฟ,สั่ง
6,เมนู,ราคา,เมนู
7,สั่ง,เมนู,ชา
8,อาหาร,อาหาร,อาหาร
9,ราคา,ชา,กาแฟ


In [28]:
df.to_excel(r'Topic_Modeling.xlsx', index = False)